In [1]:
#se importan los paquetes necesarios
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
from ipywidgets import interact
from sympy.solvers import solve, linsolve
from sympy import Symbol
from sympy import symarray
from itertools import combinations

In [2]:
#se crea la función que calculará r efectiva
def f_reff(i,x,params):
    r,b,c = params
    
    N=len(r)
        
    r_eff = r[i] 
    for j in range(N):
        r_eff = r_eff + b[i][j]*x[j]
        
    return r_eff

In [3]:
#encontrar las soluciones estacionarias de las ecuaciones de r_eff y las combinaciones de x=1/c
def solut_fact_reff_Nsp2(params):
    r,b,c = params  #se introducen los parámetros en el modelo
    N=len(r)
    
    if any(c[i]==0 for i in range(N)):  #se comprueba que ningun valor de c sea 0
        print('c=0')
        return

    x=symarray("x", N)
    #se generan listas donde posteriormente se almacenarán diversos datos de interés
    r_ef=[]
    x_n=[]
    dx_n=[]
    sols=[]
    Sols=[]
    Sols_dict={}
    ind_cc_dict={}  # diccionario que contiene 0/1 para cada combinación, siendo 1: variable=1/c
    
    for i in range(N):
        r_eff = r[i] 
        for j in range(N):
            r_eff = r_eff + b[i][j]*x[j]  #se calcula r efectiva
        dx = r_eff * x[i]*(1-c[i]*x[i])   #se crea la ecuación con esa r efectiva
        r_ef.append(r_eff)  #se almacena la r efectiva
        dx_n.append(dx)   #se almacena toda la ecuación para esa especie
        x_n.append(x[i])  #se guarda la especie que ese esta estudiando, por ejemplo x_1
    
    sol_cc = [1./c[i] for i in range(N)] #se calculan las carrying capacities 
        
    for nfix in range(0,N): #Se evaluarán las combinaciones posibles cogiendo desde 1 especie hasta N-1 especies, que será el total         
        comb_fix = np.math.factorial(N)/(np.math.factorial(nfix)*np.math.factorial(N-nfix))
        combinat = list(combinations(range(N),nfix)) #se generan las combinaciones de 0 y 1 para nfix, siendo 1 las especies que están en carrying capacity
        Sols=[]
        ind_cc=[]
        for list_fix in combinat:           
            i_cc=[]        
            Eqs=[]
            Xs=[]        
            i_cc = [ 1 if i in list_fix else 0 for i in range(N)]
            for i in range(N):
                if i not in list_fix: #se calculan las ecuaciones de las variables que no están en carrying capacity
                    eq = r_ef[i]
                    for k in list_fix:
                        eq = eq.subs({x[k]:sol_cc[k]})
                    Eqs.append(eq) #se almacena la ecuación generada
                    Xs.append(x[i])                  
                        
            sol_lin = next(iter(linsolve(Eqs,Xs))) #se resuelven las ecuaciones
            sol=[]
            for s in sol_lin:
                sol.append(s)  #se almacenan las soluciones
        
            if sol: #se generará la solución completa, mezclando las soluciones calculadas y las que estan en carrying capacity
                j = 0
                i_cc = [0 for i in range(N)]
                X_fin=[]
                for i in range(N):                    
                    if i in list_fix:
                        X_fin.append(sol_cc[i])
                        i_cc[i]=1  #está en carrying capacity
                    else:
                        X_fin.append(np.float(sol[j]))
                        j +=1
                        i_cc[i]=0  #no está en carrying capacity
                        
                Sols.append(X_fin)  #se almacena la solución entera
                ind_cc.append(i_cc) #se almacená el vector con las especies que están en carrying capacity
                                
        Sols_dict[nfix]= Sols                                
        ind_cc_dict[nfix] = ind_cc
        
    Sols_dict[N] = [ [sol_cc[i] for i in range(N)] ]
    ind_cc_dict[N] = [[1 for i in range(N)]]
    
    Sols_uniq_dict={}
    ind_cc_uniq_dict={}
    for nfix in range(0,N+1):
        ss=0
        Sols_uniq=[]
        ind_cc_uniq=[]
        for s in Sols_dict[nfix]:
            if s not in Sols_uniq:    #se eliminan posiibles soluciones repetidas        
                if all([s[i]>0 for i in range(N) if len(s)>1]):  #se comprueba que todos los valores de la solción sean mayores que 0
                    Sols_uniq.append(s)
                    ind_cc_uniq.append(ind_cc_dict[nfix][ss])
            ss +=1
            
        Sols_uniq_dict[nfix]=Sols_uniq   #lista de soluciones con todos sus valores por encima de 0
        ind_cc_uniq_dict[nfix]=ind_cc_uniq   #lista de vectores que indican qué especies estan en la carrying capacity en cada solución
        #print('nfix: ',nfix)
        #print('Sols_unique_dict: ', Sols_uniq_dict[nfix])
        #print('ind_cc_uniq_dict: ', ind_cc_uniq_dict[nfix])
    
    return(Sols_uniq_dict, ind_cc_uniq_dict)

In [4]:
#función que calcula la matriz Jacobiana cuando ninguna especie está en carrying capacity
def Jacobian_fact_X0_Nsp(X,params):
    r,b,c = params
    
    N=len(r)
    JJ = np.ndarray(shape=(N,N), dtype=float)
    
    for i in range(N):
        for j in range(N):
            JJ[i][j] = b[i][j]*X[i]*(1. - c[i]*X[i]) #se calcula cada valor de la matriz
    
    return JJ

In [5]:
#función que calcula la matriz Jacobiana cuando todas las especies están en carrying capacity
def Jacobian_fact_1C_Nsp(X,params):
    r,b,c = params
    
    N=len(r)
    x_c= np.zeros((N,))
    r_eff = np.zeros((N,))

    for i in range(N):
        r_eff[i]=f_reff(i,X,params)
        
    JJ = np.zeros(shape=(N,N), dtype=float) #se genera la matriz con todos los valores como 0
    for i in range(N):
            JJ[i][i] = -r_eff[i]  #se calcula la diagonal de la matriz que será la única que no sea 0
    return JJ

In [6]:
#función que calcula la matriz Jacobiana cuando hay mezcla de especies en carrying capacity y especies que no.
def Jacobian_fact_1C_X_Nsp(X,cc,params):
    r,b,c = params
    
    N=len(r)
    JJ_cc = np.zeros(shape=(N,N), dtype=float)  #se genera una matriz con todos los valores como 0
    for i in range(N): #para cada fila se estudia cada caso
        if cc[i]: #en carrying capacity
            for j in range(N):
                if j == i:
                    JJ_cc[i][j] = -f_reff(j,X,params)  #se calcula el valor para esa posición
                    
        else:  #no en carrying capacity
            for j in range(N):
                JJ_cc[i][j] = b[i][j]*X[i]*(1.-c[i]*X[i]) #se calcula el valor para esa posición

    return JJ_cc

In [7]:
#función que permite calcular los autovectores de cada solución
def lambdas_fact_X_icc_Nsp(X,i_cc,params):
    list_0 = []
    list_1 = []
    for i in range(len(params[0])):
        list_0.append(0) 
        list_1.append(1)
    if i_cc == list_0:  #ninguna especie esta en carrying capacity
        JJ = Jacobian_fact_X0_Nsp(X,params)
    elif i_cc == list_1:    #todas las especies estan en carrying capacity    
        JJ = Jacobian_fact_1C_Nsp(X,params)
    else:   #mezcla de especies en carrying capacity
        JJ = Jacobian_fact_1C_X_Nsp (X,i_cc,params)
  

    lambdas=np.linalg.eigvals(JJ) #se calculan los autovalores
    return lambdas

In [8]:
#se introducen los parámetros deseados, en este caso son los de los OTUs 2,5 y 6 ya modificados para ser estables
r=[0.25, 0.6, 0.5]
b=[[0.04,-0.18,0.01], [0.12,-0.22,0.19], [0.1,0.36,-0.29]]
c=[0.0001, 0.0001,0.0001]
N= len(r)    
params=(r,b,c)
sols_fact_dict, sols_cc_dict = solut_fact_reff_Nsp2(params) #se calculan las soluciones estacionarias

In [9]:
X_finite=[]
Xss=[]
Lss=[]
print(sols_cc_dict)
for (ns,list_sols),(ni,list_cc) in zip(sols_fact_dict.items(), sols_cc_dict.items()):
    if list_sols:        
        print('n_cc:', ns, 'n_sols:', len(list_sols))
        for X,i_cc in zip(list_sols,list_cc): #se barre la lista de soluciones y de vectores de carrying capacity
            print('X:',X, 'i_cc:',i_cc)
            X_finite.append(X)
            lambdas = lambdas_fact_X_icc_Nsp(X,i_cc,params) #se calculan los autovalores
            if all(lambdas[i].real<0 for i in range(len(lambdas))): #se mira si todos tienen parte real negativa
                print('ESTABLE')
                Xss.append([X[i] for i in range(len(X))])  #se almacena la solución estable
                Lss.append([lambdas[i] for i in range(len(lambdas))])    #se almacena qué especies están en carrying capacity en esa solución

{0: [], 1: [[0, 1, 0]], 2: [[1, 1, 0], [1, 0, 1], [0, 1, 1]], 3: [[1, 1, 1]]}
n_cc: 1 n_sols: 1
X: [38565.27777777777, 10000.0, 25713.88888888889] i_cc: [0, 1, 0]
n_cc: 2 n_sols: 3
X: [10000.0, 10000.0, 15863.793103448277] i_cc: [1, 1, 0]
X: [10000.0, 14093.636363636364, 10000.0] i_cc: [1, 0, 1]
X: [42493.75, 10000.0, 10000.0] i_cc: [0, 1, 1]
ESTABLE
n_cc: 3 n_sols: 1
X: [10000.0, 10000.0, 10000.0] i_cc: [1, 1, 1]


In [12]:
#ejemplo de barrido de b14 delos OTUs 2,5 y 6 modificados
rango= np.linspace(-1,1,101,dtype=list)
for i in range(len(rango)):
    rango[i]=round(rango[i],2) #se crea el rango de 100 valores entre -1 y 1 que se va a utilizar para el barrido

#f_out = open('sol_b14','a')
#f_out.write("r=[0.25, 0.6, 0.5, 0.35]\nb=[[0.04,-0.18,0.01,i], [0.12,-0.22,0.19,0], [0.1,0.36,-0.29,0], [0,0,0,-0.01]]\nc=[0.0001,0.0001,0.0001,0.0001]\n\n")
#f_out.close()
    
for i in rango:
    #para cada valor, se seguirán los mismos pasos mostrados para el cálculo de una solución
    r=[0.25, 0.6, 0.5, 0.35]
    b=[[0.04,-0.18,0.01,i], [0.12,-0.22,0.19,0], [0.1,0.36,-0.29,0], [0,0,0,-0.01]]
    c=[0.0001, 0.0001,0.0001,0.0001]
    N= len(r)    
    params=(r,b,c)
    sols_fact_dict, sols_cc_dict = solut_fact_reff_Nsp2(params)
    Xss=[]
    l=[]
    c=[]
    X_finite=[]
    for (ns,list_sols),(ni,list_cc) in zip(sols_fact_dict.items(), sols_cc_dict.items()):
        for X,i_cc in zip(list_sols,list_cc):
            lambdas = lambdas_fact_X_icc_Nsp(X,i_cc,params)
            X_finite.append(X)
            if all(lambdas[i].real<0 for i in range(len(lambdas))):
                Xss.append(X)
                l.append(lambdas)
                c.append(i_cc)
    if len(Xss) != 0:
        #f_out = open('sol_b14','a')
        for i in range(len(Xss)):
            print(b[0][3])  #valor dado al parámetro bij
            print(sum(Xss[i]))  #total de población
            print(c[i])  #especies en carrying capacity
            print(max(l[i].real))  #autovalor menos negativo
            print(min(l[i].real))  #autovalor más negativo
            print('\n')
       
            #f_out.write(str(b[1][3])+ '\t')
            #f_out.write(str(sum(Xss[i]))+'\t')
            #f_out.write(str(c[i])+'\t')
            #f_out.write(str(max(l[i].real))+'\t')
            #f_out.write(str(min(l[i].real))+'\n')
        #f_out.close()
        

0.0
62528.75
[0, 1, 1, 0]
-0.34877500000000006
-5523.125156250001


0.02
67493.75
[0, 1, 1, 1]
-100.35
-4449.875


0.04
62493.75
[0, 1, 1, 1]
-300.35
-3949.875


0.06
57493.75
[0, 1, 1, 1]
-500.35
-3449.875


0.08
52493.75
[0, 1, 1, 1]
-700.35
-2949.875


0.1
47493.75
[0, 1, 1, 1]
-524.37515625
-2449.875


0.12
42493.75
[0, 1, 1, 1]
-124.62515625000006
-1949.875


0.14
40000.0
[1, 1, 1, 1]
-100.25000000000023
-1700.5


0.16
40000.0
[1, 1, 1, 1]
-300.25
-1700.5


0.18
40000.0
[1, 1, 1, 1]
-500.25
-1700.5


0.2
40000.0
[1, 1, 1, 1]
-700.25
-1900.35


0.22
40000.0
[1, 1, 1, 1]
-900.25
-2100.35


0.24
40000.0
[1, 1, 1, 1]
-900.5999999999999
-2300.35


0.26
40000.0
[1, 1, 1, 1]
-900.5999999999999
-2500.35


0.28
40000.0
[1, 1, 1, 1]
-900.5999999999999
-2700.3500000000004


0.3
40000.0
[1, 1, 1, 1]
-900.5999999999999
-2900.35


0.32
40000.0
[1, 1, 1, 1]
-900.5999999999999
-3100.35


0.34
40000.0
[1, 1, 1, 1]
-900.5999999999999
-3300.3500000000004


0.36
40000.0
[1, 1, 1, 1]
-900.599999999999